In [1]:
import pandas as pd
import numpy as np
import random
import joblib
import seaborn as sns
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import label_binarize, LabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report, roc_curve, auc
#import graphviz
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

#### The data

In [148]:
#df = pd.read_csv('../../data/more_features/more_feats_correlated_noisy_4_missing_1.csv')
train_df = pd.read_csv('../../final/data/train_set_noisy_4_missing_5.csv')
train_df = train_df.fillna(-1)
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat
0,7.832577,-1.000000,3.266163,1.641831,-1.0,-1.000000,-1.0,-1.0,1,-1.000000,84.109029,70.053478,-1.0,2.258603,-1.0,23.497731,-1.0
1,12.434184,-1.000000,4.482646,-1.000000,-1.0,103.843580,-1.0,-1.0,1,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.0
2,6.514719,-1.000000,-1.000000,2.563764,-1.0,-1.000000,-1.0,-1.0,0,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,19.544156,-1.0
3,8.622178,-1.000000,-1.000000,0.000000,-1.0,101.981112,-1.0,-1.0,1,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.0,25.866535,-1.0
4,10.725831,14.337773,0.885657,6.654129,-1.0,77.347657,-1.0,-1.0,1,0.472443,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,-1.0


In [149]:
# class_dict = constants.CLASS_DICT
# df['label'] = df['label'].replace(class_dict)
# X = df.iloc[:, 0:-1]
# y = df.iloc[:, -1]
# X_train, X_test_fake, y_train, y_test_fake = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)

In [150]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]
X_test.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat
0,11.783006,112.011007,-1.000000,3.308648,348.014906,86.577465,-1.000000,4.082935,0,1.617594,131.868365,-1.000000,-1.000000,-1.000000,-1.000000,35.349018,-1.000000
1,12.609666,104.475099,5.445972,-1.000000,-1.000000,75.082198,-1.000000,5.038345,1,0.334722,97.568987,83.161880,-1.000000,7.147270,-1.000000,37.828999,-1.000000
2,7.940172,4.925400,3.255130,5.766738,209.031102,75.254082,-1.000000,3.165345,1,1.649478,2.082970,-1.000000,-1.000000,-1.000000,-1.000000,23.820517,-1.000000
3,7.785840,103.757096,2.874384,3.839597,-1.000000,81.873581,39.638010,2.852876,1,0.637843,127.412065,38.534760,51.437387,28.150784,73.296289,23.357519,-1.000000
4,12.359715,51.274141,0.178269,1.233844,474.589534,76.469181,49.839768,4.848900,1,0.621034,137.433107,55.314614,47.302738,3.808612,60.076211,37.079144,10.501658


In [151]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [152]:
dt = DecisionTreeClassifier(random_state=constants.SEED).fit(X_train, y_train)
y_pred = dt.predict(X_test)
test_df = pd.DataFrame()
test_df['y_actual'] = y_test
test_df['y_pred'] = y_pred
test_df.isna().sum()

C:\Users\User\Anaconda3\lib\site-packages\sklearn\tree\tree.py:163: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



y_actual    0
y_pred      0
dtype: int64

In [153]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

72.64285714285714

In [154]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.7264285714285714, 0.6942581290325625, 0.8279144897167939)

In [155]:
test_df.y_pred.unique()

array([6., 3., 4., 2., 0., 1., 5., 7.])

In [156]:
test_df.y_actual.unique()

array([7, 3, 4, 5, 2, 6, 1, 0], dtype=int64)

In [157]:
test_df.to_csv('../../final/test_dfs/dt_noisy_4_missing_5.csv', index=False)

#### Without inconclusive diagnosis

In [158]:
test_df_sans_inconc = test_df[~((test_df.y_actual == 7.0) | (test_df.y_pred ==7.0))]

In [159]:
test_df_sans_inconc.y_actual.unique()

array([3, 4, 5, 2, 6, 1, 0], dtype=int64)

In [160]:
test_df_sans_inconc.y_pred.unique()

array([3., 4., 2., 0., 6., 1., 5.])

In [161]:
success_rate, success_df = utils.success_rate(test_df_sans_inconc)
success_rate

77.55465963715305

#### Feature Importance

In [17]:
# utils.compute_feature_importance(dt, X_train)

#### Using SHAP

In [18]:
# import shap

##### delete from here

In [19]:
# shap.initjs()

# #background = X_train.iloc[0:100,:].values.astype('float')
# background = X_train.values.astype('float')
# explainer = shap.KernelExplainer(dt.predict_proba, background)

# all_shap_values = explainer.shap_values(X_test)

In [20]:
# shap.force_plot(explainer.expected_value[6], row_shap_values[6], row_for_prediction, feature_names=X_train.columns)

In [21]:
# row_for_prediction = X_test.iloc[1:2,:].values.astype('float')
# row_shap_values = explainer.shap_values(row_for_prediction)
# shap.summary_plot(row_shap_values, row_for_prediction, feature_names = list(X_test.columns))

In [22]:
# len(list(X_test.columns))

In [23]:
# shap.summary_plot(shap_values, X_test)

In [24]:
# shap_values[0]

##### end here

In [25]:
# explainer = shap.TreeExplainer(dt)
# shap_values = explainer.shap_values(X_test)
# len(shap_values)

In [26]:
# shap_values[0].shape

In [27]:
# shap.summary_plot(shap_values, X_test)

In [28]:
# def get_ABS_SHAP(df_shap,df):
#     #import matplotlib as plt
#     # Make a copy of the input data
#     shap_v = pd.DataFrame(df_shap)
#     feature_list = df.columns
#     shap_v.columns = feature_list
#     df_v = df.copy().reset_index().drop('index',axis=1)
    
#     # Determine the correlation in order to plot with different colors
#     corr_list = list()
#     for i in feature_list:
#         b = np.corrcoef(shap_v[i],df_v[i])[1][0]
#         corr_list.append(b)
#     corr_df = pd.concat([pd.Series(feature_list),pd.Series(corr_list)],axis=1).fillna(0)
 
#     # Make a data frame. Column 1 is the feature, and Column 2 is the correlation coefficient
#     corr_df.columns  = ['Variable','Corr']
#     corr_df['Sign'] = np.where(corr_df['Corr']>0,'red','blue')
    
#     shap_abs = np.abs(shap_v)
#     k=pd.DataFrame(shap_abs.mean()).reset_index()
#     k.columns = ['Variable','SHAP_abs']
#     k2 = k.merge(corr_df,left_on = 'Variable',right_on='Variable',how='inner')
#     k2 = k2.sort_values(by='SHAP_abs',ascending = True)
    
#     k2_f = k2[['Variable', 'SHAP_abs', 'Corr']]
#     k2_f['SHAP_abs'] = k2_f['SHAP_abs'] * np.sign(k2_f['Corr'])
#     k2_f.drop(columns='Corr', inplace=True)
#     k2_f.rename(columns={'SHAP_abs': 'SHAP'}, inplace=True)
    
#     return k2_f


In [29]:
# foo_all = pd.DataFrame()

# for k,v in list(enumerate(dt.classes_)):
#     foo = get_ABS_SHAP(shap_values[k], X_test)
#     foo['class'] = v
#     foo_all = pd.concat([foo_all,foo])

# import plotly_express as px
# px.bar(foo_all,x='SHAP', y='Variable', color='class')

#### Using LIME

In [30]:
# import lime
# from lime import lime_tabular

In [31]:
# lime_explainer = lime_tabular.LimeTabularExplainer(training_data=np.array(X_train), feature_names=X_train.columns, 
#                                               class_names=list(class_dict.keys()), mode='classification')

In [32]:
# row_for_prediction = row_for_prediction.reshape(18,)

In [33]:
# exp = lime_explainer.explain_instance(row_for_prediction, dt.predict_proba)
# exp.show_in_notebook(show_table=True)

In [34]:
# y_test.iloc[1:2]

In [35]:
# exp = explainer.explain_instance(data_row=X_test.iloc[1],  predict_fn=dt.predict_proba, num_features=5)
# exp.show_in_notebook(show_table=True)

In [36]:
# X_test.iloc[39]